В данном задании пытался предсказать стоимость индекса ММВБ на основе 5 летних исторических данных по сырой нефти BRENT и USD относительно рубля (стоимость на момент закрытия, внутридневные объемы). Месяц август 2019 был взят как предсказанный период и в обучающей выборке отсутствовал. Если честно, то результаты не порадовали, возможно они хорошо коррелируются, но для торгов совсем не подходят. Как ошибки вижу следующее:
- маленький объем выборки, всего 1000+ дневных позиций, можно перейти на часовые отсечки, что существенно увеличит поле для обучения
- полное отсутствие индикаторов тех анализа (различные скользящие средняя, и еще куча подобных), что при должном применении также могли внести положительный вклад в результат.
- создание предсказания на длительный период (почти месяц), наверное следует анализировать лишь следующий рассматриваемый период к анализируемому, те в моем случае 1 день.

In [1]:
import pandas as pd
import numpy as np
from datetime import date
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
plt.style.use('fivethirtyeight')

In [3]:
study_DB=pd.read_csv(r'data_to_work\study.csv')
study_DB.columns

Index(['Unnamed: 0', 'MOEX', 'DATE', 'PRICE_MOEX', 'VOL_MOEX', 'BRENT',
       'PRICE_BRENT', 'VOL_BRENT', 'RTS', 'PRICE_RTS', 'VOL_RTS', 'USD',
       'PRICE_USD', 'VOL_USD'],
      dtype='object')

In [4]:
target=study_DB['PRICE_MOEX']
features=study_DB[['PRICE_BRENT','VOL_BRENT', 'PRICE_USD', 'VOL_USD']]

модель LR

In [5]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(features,target,test_size=0.3, random_state=42)

from sklearn.linear_model import LinearRegression as LR

from sklearn.ensemble import RandomForestRegressor as RF

model1=LR()

model1.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [6]:
y_pred = model1.predict(X_test)

from sklearn.metrics import r2_score as r2, mean_squared_error as mse

r2(y_test, y_pred)

0.6210518506127733

Model2 RF

In [7]:

model2=RF(n_estimators=150, max_depth=15, min_samples_leaf=4) 

model2.fit(X_train,y_train)

y_pred_2 = model2.predict(X_test)

r2(y_test, y_pred_2)

0.8526479511711621

взял за основу вторую модель, так как результаты R2 довольно хорошие.

In [8]:
test=pd.read_csv(r'data_to_work\target.csv')
test.head()

test.loc[:,['PRICE_BRENT', 'VOL_BRENT', 'PRICE_USD', 'VOL_USD']]

y_pred_test= model2.predict(test. loc[:,['PRICE_BRENT', 'VOL_BRENT', 'PRICE_USD', 'VOL_USD']])

result = pd.concat([test['PRICE_MOEX'],pd.Series(y_pred_test),], axis=1).rename(columns={0:'Predicted Price'})

result

,PRICE_MOEX,Predicted Price
0,2729.42,2706.113082
1,2674.90,2424.243892
2,2649.16,2423.102130
3,2683.35,2146.645735
4,2674.98,2061.844880
5,2696.69,2076.753150
6,2679.71,2040.870787
7,2689.93,2023.011958
8,2680.12,2632.341975
9,2637.04,2175.878378
